## 加载模型

In [1]:
model_name_or_path = 'THUDM/chatglm3-6b-128k'

# Lora 相关的超参数
lora_rank = 64                     
lora_alpha = 64                    
lora_dropout = 0.1                 

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, 
                                          trust_remote_code=True)

In [3]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                            quantization_config=q_config,
                                            device_map='auto',
                                            trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/home/da_Ops01/anaconda3/envs/llm/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
input_text = "请问你是谁？"
response, history = model.chat(tokenizer=tokenizer, query=input_text, history=[])
print(response)
print(history)

我是一个名为智谱清言的人工智能助手，可以叫我小智🤖，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。
[{'role': 'user', 'content': '请问你是谁？'}, {'role': 'assistant', 'metadata': '', 'content': '我是一个名为智谱清言的人工智能助手，可以叫我小智🤖，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。'}]


In [5]:
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
from peft import TaskType, LoraConfig

target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [6]:
from peft import get_peft_model, prepare_model_for_kbit_training
kbit_model = prepare_model_for_kbit_training(model)
lora_model = get_peft_model(kbit_model, lora_config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [7]:
lora_model.print_trainable_parameters()

trainable params: 15,597,568 || all params: 6,259,181,568 || trainable%: 0.24919500785441986


# 数据处理

In [8]:
from datasets import Dataset
import pandas as pd

excel_file_path = 'data/chatglm-summary-0310.xlsx'
df = pd.read_excel(excel_file_path)
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['content', 'summary'],
    num_rows: 1101
})


In [9]:
# 初始化一个字典来保存每一列的最大长度和对应的值
max_length_values = {feature: {"max_length": 0, "value": ""} for feature in dataset.features}

# 遍历数据集中的每一行
for row in dataset:
    # 对于每一列，更新最大长度和对应的值
    for feature in dataset.features:
        current_length = len(row[feature])
        if current_length > max_length_values[feature]["max_length"]:
            max_length_values[feature]["max_length"] = current_length
            max_length_values[feature]["value"] = row[feature]

# 打印每一列的最大长度和对应的值
for feature, info in max_length_values.items():
    print(f"特征 '{feature}' 的最大长度: {info['max_length']}, 对应的值: {info['value'][:100]}")
    print('----------')

特征 'content' 的最大长度: 32767, 对应的值: 这时，她已知道父亲时时往伦敦是为着什么。
   

    有一个人在那个城市。
   

    而且，那人逐渐嚣张，电话电讯时时传到胡家。
   

    一日，直子告诉胡球：“我将随向先生到伦
----------
特征 'summary' 的最大长度: 948, 对应的值: 法国现代文学史上的一些重要作家包括塞维尼夫人、博马舍、大仲马、梅里美、乔治·桑、福楼拜、龚古尔兄弟、凡尔纳、左拉、都德、法朗士、兰波、罗兰、纪德、普鲁斯特、莫泊桑、莫洛亚、朱林·格林、萨特、加缪等。这
----------


In [10]:
# 确定下最大长度
q_ids = tokenizer.encode(text=max_length_values['content']['value'], add_special_tokens=False)
a_ids = tokenizer.encode(text=max_length_values['summary']['value'], add_special_tokens=False)
print(len(q_ids))
print(len(a_ids))

21308
658


In [11]:
max_input_length = 22000                    
max_output_length = 800
prompt_text = """以下是来自于某本书中的单章节内容，请你使用「平铺直叙」的方式总结下本章节的内容
---------以下是章节内容---------
"""

In [12]:
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    example['content'] = example['content'].replace('\n', '')
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [13]:
column_names = dataset.column_names
tokenized_dataset = dataset.map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

In [14]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1101
})

In [15]:
tokenized_dataset.select(range(1000))

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1000
})

# 准备训练

In [16]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = max_output_length + max_input_length, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [17]:
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=f"models/{model_name_or_path}_0310",          # 输出目录
    per_device_train_batch_size=2,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=4,                     # 梯度累积步数
    learning_rate=1e-5,                                # 学习率
    num_train_epochs=5,                                # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.05,                                  # 预热比例
    logging_steps=50,                                 # 日志记录步数
    save_strategy="epoch",                             # 模型保存策略
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练

)

In [19]:
trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
trainer.train()

/home/da_Ops01/anaconda3/envs/llm/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,1.898000
100,1.790200
150,1.703500
200,1.661100
250,1.665700
300,1.616400
350,1.608500
400,1.581600
450,1.585100
500,1.576200


/home/da_Ops01/anaconda3/envs/llm/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/da_Ops01/anaconda3/envs/llm/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/da_Ops01/anaconda3/envs/llm/lib/python

TrainOutput(global_step=685, training_loss=1.6314939178689554, metrics={'train_runtime': 18749.1608, 'train_samples_per_second': 0.294, 'train_steps_per_second': 0.037, 'total_flos': 1.1261881945405686e+18, 'train_loss': 1.6314939178689554, 'epoch': 4.97})

In [21]:
trainer.model.save_pretrained(f"models/{model_name_or_path}_0310")